Set Shots - Expected Miss Model - Model Tuning and Building - GBM

In [10]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/")

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_score_model.config import set_shots_file_path
# from expected_score_model.domain.preprocessing.data_preprocessor import DataPreprocessor
# from expected_score_model.domain.contracts.mappings import Mappings
from expected_score_model.domain.contracts.modelling_data_contract import ModellingDataContract
from expected_score_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from expected_score_model.domain.modelling.supermodel import SuperXGBClassifier
from expected_score_model.domain.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Model Version

In [12]:
model_version = 1
model_name = 'expected_miss_set'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/predictions/"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/"

In [13]:
RESPONSE = ModellingDataContract.RESPONSE_MISS

Load Data

In [14]:
df_modelling = pd.read_csv(set_shots_file_path)
df_modelling.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Miss,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,random100,ModellingFilter
11052,208,possGain,behind,1663.0,4,634,683.0,Geelong,Geelong,Tom Hawkins,Tom_Hawkins,Kick,41.0,25.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,0,1,0,1,1,0,105.0,32.0,73.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,41.0,48.0,48.0,26.0,25.0,21.0,21.0,14.0,683.0,644.0,642.0,641.0,49.0,39.0,8.062258,39.0,25.0,46.324939,0.570040,32.660913,0.116867,6.695990,0.364585,20.889203,1521.0,3.663562,True,43,True
11053,212,kickIn,goal,1730.0,4,787,885.0,Geelong,Geelong,Patrick Dangerfield,Patrick_Dangerfield,Kick,51.0,43.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,107.0,33.0,74.0,Kick,Contested Mark,Contest Target,Kick Inside 50 Result,51.0,53.0,53.0,53.0,43.0,38.0,38.0,38.0,885.0,852.0,851.0,850.0,98.0,33.0,5.385165,29.0,43.0,51.865210,0.977446,56.003541,0.069260,3.968302,0.214332,12.280314,841.0,3.367296,True,29,True
11054,213,centreBounce,goal,1754.0,4,956,979.0,Sydney,Sydney,Paddy McCartin,Paddy_McCartin,Kick,66.0,0.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,0,6,119.0,33.0,86.0,Kick,Free For,Kick Inside 50 Result,Kick Into F50,66.0,74.0,74.0,27.0,0.0,0.0,0.0,17.0,979.0,977.0,968.0,967.0,23.0,2.0,8.000000,14.0,0.0,14.000000,0.000000,0.000000,0.482343,27.636207,2.588598,148.315731,196.0,2.639057,True,78,True
11055,219,centreBounce,goal,1846.0,4,1252,1266.0,Sydney,Sydney,Chad Warner,Chad_Warner,Kick,26.0,3.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,0,6,125.0,39.0,86.0,Kick,Uncontested Mark,Ground Kick,Handball,26.0,18.0,-5.0,-2.0,3.0,2.0,-4.0,3.0,1266.0,1265.0,1264.0,1260.0,14.0,1.0,8.062258,54.0,3.0,54.083269,0.055499,3.179830,0.118569,6.793500,0.365993,20.969862,2916.0,3.988984,True,53,True
11056,228,centreBounce,goal,1912.0,4,1519,1560.0,Geelong,Geelong,Jeremy Cameron,Jeremy_Cameron,Kick,41.0,21.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,131.0,45.0,86.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,41.0,49.0,49.0,25.0,21.0,17.0,17.0,33.0,1560.0,1535.0,1532.0,1531.0,41.0,25.0,8.944272,39.0,21.0,44.294469,0.493941,28.300756,0.127885,7.327246,0.400462,22.944787,1521.0,3.663562,True,66,True


In [15]:
training_data = df_modelling[df_modelling[ModellingDataContract.TRAIN_TEST_SPLIT_COL]]
test_data = df_modelling[~df_modelling[ModellingDataContract.TRAIN_TEST_SPLIT_COL]]

In [16]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]

In [17]:
X_test.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,random100,ModellingFilter
11,122,possGain,goal,1074.0,3,28,72.0,Sydney,Sydney,Nick Blakey,Nick_Blakey,Kick,28.0,-14.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,0,6,51.0,52.0,-1.0,Kick,Uncontested Mark,Kick Inside 50 Result,Kick Into F50,28.0,35.0,35.0,32.0,-14.0,-12.0,-12.0,35.0,72.0,38.0,36.0,36.0,44.0,34.0,7.280110,50.0,14.0,51.923020,0.273009,15.642246,0.119147,6.826615,0.368686,21.124166,2500.0,3.912023,True,83,False
12,132,possGain,goal,1168.0,3,422,469.0,Sydney,Sydney,Logan McDonald,Logan_McDonald,Kick,47.0,-4.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,0,6,57.0,58.0,-1.0,Kick,Mark On Lead,Kick,Gather,47.0,55.0,30.0,22.0,-4.0,-4.0,2.0,6.0,469.0,434.0,432.0,430.0,47.0,35.0,8.000000,31.0,4.0,31.256999,0.128323,7.352379,0.205222,11.758330,0.672664,38.540807,961.0,3.433987,True,93,False
15,144,throwIn,goal,1283.0,3,1014,1041.0,Sydney,Sydney,Callum Mills,Callum_Mills,Kick,29.0,-3.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,0,6,58.0,71.0,-13.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,29.0,37.0,37.0,22.0,-3.0,-2.0,-2.0,-16.0,1041.0,1019.0,1018.0,1017.0,27.0,22.0,8.062258,49.0,3.0,49.091751,0.061148,3.503532,0.130680,7.487397,0.405895,23.256079,2401.0,3.891820,True,99,False
20,194,throwIn,goal,1678.0,4,589,654.0,Sydney,Sydney,Luke Parker,Luke_Parker,Kick,49.0,-43.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,0,6,71.0,107.0,-36.0,Kick,Uncontested Mark,Kick Inside 50 Result,Kick Into F50,49.0,53.0,53.0,7.0,-43.0,-40.0,-40.0,-51.0,654.0,624.0,622.0,621.0,65.0,30.0,5.000000,29.0,43.0,51.865210,0.977446,56.003541,0.069260,3.968302,0.214332,12.280314,841.0,3.367296,True,96,False
31,84,centreBounce,goal,676.0,2,418,472.0,Western Bulldogs,Western Bulldogs,Lachlan McNeil,Lachlan_McNeil,Kick,71.0,0.0,effective,True,NaN,141,160,Collingwood,Western Bulldogs,right,202101_Collingwood_WesternBulldogs,202101,2021,1,0,6,0,6,14.0,26.0,-12.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,71.0,42.0,42.0,7.0,0.0,-21.0,-21.0,-20.0,472.0,430.0,428.0,427.0,54.0,42.0,35.805028,9.0,0.0,9.000000,0.000000,0.000000,0.814019,46.639866,-15.483871,-887.160457,81.0,2.197225,True,95,False


Preprocess Data

In [18]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train_preproc = preprocessor.transform(X_train)
# X_test_preproc = preprocessor.transform(X_test)

In [19]:
X_train_preproc = X_train[ModellingDataContract.modelling_feature_list]
X_test_preproc = X_test[ModellingDataContract.modelling_feature_list]

In [20]:
X_train_preproc.shape, X_test_preproc.shape

((8749, 2), (2308, 2))

In [21]:
X_train_preproc.head()

,Distance_to_Middle_Goal,Angle_to_Middle_Goal
0,52.009614,0.908067
1,43.600459,0.931882
2,36.674242,0.451453
3,33.015148,0.957589
4,46.572524,0.260602


In [22]:
X_test_preproc.head()

,Distance_to_Middle_Goal,Angle_to_Middle_Goal
11,51.923020,0.273009
12,31.256999,0.128323
15,49.091751,0.061148
20,51.865210,0.977446
31,9.000000,0.000000


Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [39]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train, monotonicity_constraints = ModellingDataContract.monotone_constraints_miss)

In [40]:
xgb_tuner.training_data.head()

,Distance_to_Middle_Goal,Angle_to_Middle_Goal
0,52.009614,0.908067
1,43.600459,0.931882
2,36.674242,0.451453
3,33.015148,0.957589
4,46.572524,0.260602


In [41]:
xgb_tuner.tune_hyperparameters()

[I 2023-06-28 15:30:30,365] A new study created in memory with name: no-name-5cb2ae15-ba09-4ae8-bf33-e53d1c1ba648
[I 2023-06-28 15:30:30,410] Trial 0 finished with value: 0.4350398554399821 and parameters: {'max_depth': 18, 'min_child_weight': 56, 'eta': 0.08111868519373076, 'gamma': 0.0006496527088838824, 'lambda': 2.8028471554776995, 'alpha': 0.0007984462750542095, 'subsample': 0.5133767761298663, 'colsample_bytree': 0.3315258853479373}. Best is trial 0 with value: 0.4350398554399821.
[I 2023-06-28 15:30:30,433] Trial 1 finished with value: 0.47165840310918394 and parameters: {'max_depth': 11, 'min_child_weight': 85, 'eta': 0.06154235802986995, 'gamma': 0.000261675310244247, 'lambda': 0.27066708226365616, 'alpha': 0.0006371433569365175, 'subsample': 0.33207297013809506, 'colsample_bytree': 0.21514687535991134}. Best is trial 0 with value: 0.4350398554399821.
[I 2023-06-28 15:30:30,456] Trial 2 finished with value: 0.30006049853340133 and parameters: {'max_depth': 10, 'min_child_weigh

Number of finished trials:  100
Best trial:
  Value: 0.294486198678062
  Params: 
    max_depth: 17
    min_child_weight: 59
    eta: 0.7636716412242833
    gamma: 0.0007132888025782703
    lambda: 0.001372933843879598
    alpha: 0.0005534493773679674
    subsample: 0.5733325829256505
    colsample_bytree: 0.49245285534680117


In [42]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 17,
 'min_child_weight': 59,
 'eta': 0.7636716412242833,
 'gamma': 0.0007132888025782703,
 'lambda': 0.001372933843879598,
 'alpha': 0.0005534493773679674,
 'subsample': 0.5733325829256505,
 'colsample_bytree': 0.49245285534680117}

Training Model - SuperXGBClassifier class for training and predictions

In [43]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = ModellingDataContract.monotone_constraints_miss

In [44]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test, 
                               params = params)

In [45]:
super_xgb.fit()

[0]	validation_0-logloss:0.41135	validation_1-logloss:0.41983
[1]	validation_0-logloss:0.36779	validation_1-logloss:0.37995
[2]	validation_0-logloss:0.36021	validation_1-logloss:0.37429
[3]	validation_0-logloss:0.32119	validation_1-logloss:0.33623
[4]	validation_0-logloss:0.31537	validation_1-logloss:0.33117
[5]	validation_0-logloss:0.31440	validation_1-logloss:0.32994
[6]	validation_0-logloss:0.31012	validation_1-logloss:0.32456
[7]	validation_0-logloss:0.31010	validation_1-logloss:0.32476
[8]	validation_0-logloss:0.30945	validation_1-logloss:0.32378
[9]	validation_0-logloss:0.30948	validation_1-logloss:0.32329
[10]	validation_0-logloss:0.30950	validation_1-logloss:0.32393
[11]	validation_0-logloss:0.30949	validation_1-logloss:0.32389
[12]	validation_0-logloss:0.30935	validation_1-logloss:0.32339
[13]	validation_0-logloss:0.30938	validation_1-logloss:0.32377
[14]	validation_0-logloss:0.30939	validation_1-logloss:0.32383
[15]	validation_0-logloss:0.30945	validation_1-logloss:0.32386
[1

In [46]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.49245285534680117, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.0007132888025782703, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.7636716412242833, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=17, max_leaves=None,
              min_child_weight=59, missing=nan,
              monotone_constraints={'Angle_to_Middle_Goal': 1,
                                    'Distance_to_Middle_Goal': 1},
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [47]:
super_xgb.xgb_model.get_booster().feature_names

['Distance_to_Middle_Goal', 'Angle_to_Middle_Goal']

In [48]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [49]:
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]

Check Average Predictions

In [50]:
train_probas.mean(), training_data[RESPONSE].mean(), test_probas.mean(), test_data[RESPONSE].mean()

(0.12618619, 0.12344267916333296, 0.12469771, 0.13128249566724437)

Export model

In [51]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [52]:
train_info = training_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
train_export = pd.concat([train_info, X_train_preproc.reset_index(drop=True)], axis=1)
train_export['xgb_preds'] = train_preds
train_export['xgb_probas'] = train_probas
train_export.to_csv(prediction_output_path + 'train_predictions_' + model_file_name + '.csv', index = False)
train_export.head()

,Match_ID,Chain_Number,Order,Miss,Distance_to_Middle_Goal,Angle_to_Middle_Goal,xgb_preds,xgb_probas
0,202101_BrisbaneLions_Sydney,17,183.0,0,52.009614,0.908067,0,0.417739
1,202101_BrisbaneLions_Sydney,28,295.0,0,43.600459,0.931882,0,0.160574
2,202101_BrisbaneLions_Sydney,34,354.0,0,36.674242,0.451453,0,0.042082
3,202101_BrisbaneLions_Sydney,53,537.0,0,33.015148,0.957589,0,0.106043
4,202101_BrisbaneLions_Sydney,59,578.0,0,46.572524,0.260602,0,0.071151


In [53]:
test_info = test_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
test_export = pd.concat([test_info, X_test_preproc.reset_index(drop=True)], axis=1)
test_export['xgb_preds'] = test_preds
test_export['xgb_probas'] = test_probas
test_export.to_csv(prediction_output_path + 'test_predictions_' + model_file_name + '.csv', index = False)
test_export.head()

,Match_ID,Chain_Number,Order,Miss,Distance_to_Middle_Goal,Angle_to_Middle_Goal,xgb_preds,xgb_probas
0,202101_BrisbaneLions_Sydney,122,1074.0,0,51.923020,0.273009,0,0.177386
1,202101_BrisbaneLions_Sydney,132,1168.0,0,31.256999,0.128323,0,0.014101
2,202101_BrisbaneLions_Sydney,144,1283.0,0,49.091751,0.061148,0,0.155612
3,202101_BrisbaneLions_Sydney,194,1678.0,0,51.865210,0.977446,1,0.610772
4,202101_Collingwood_WesternBulldogs,84,676.0,0,9.000000,0.000000,0,0.007381


Save preprocessor

In [ ]:
# joblib.dump(preprocessor, preprocessor_output_path + preprocessor_file_name + ".joblib")